AUTOMATED WEB SCRAPER 


An Automated Web Scraper is a software tool designed to extract specific information from websites without manual effort. In today’s data-driven world, businesses and developers rely heavily on up-to-date online data such as product prices, news articles, job listings, and research data. Collecting such information manually is time-consuming and inefficient.This project focuses on building an automated system that fetches, processes, and stores web data in a structured format, reducing human effort and improving accuracy.


The Automated Web Scraper is a software tool designed to extract structured information from websites without manual interaction. As businesses increasingly rely on online data for decision-making, collecting information manually becomes time-consuming, repetitive, and error-prone. This project aims to build an automated system that fetches web data, processes it, and stores it in a usable format, enabling faster insights and reducing human effort. This Project is done on the features of smart phone.



In [7]:
pip install requests


Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ------------------------ --------------- 3/5 [certifi]
   -------------------------------- ------- 4/5 [requests]
   -------------------------------- ------- 4/5 [requests]
   ---------------------------------------- 5/5 [requests]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
!pip install beautifulsoup4


Defaulting to user installation because normal site-packages is not writeable

   ------------- -------------------------- 1/3 [soupsieve]
   ------------- -------------------------- 1/3 [soupsieve]
   -------------------------- ------------- 2/3 [beautifulsoup4]
   -------------------------- ------------- 2/3 [beautifulsoup4]
   -------------------------- ------------- 2/3 [beautifulsoup4]
   ---------------------------------------- 3/3 [beautifulsoup4]




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
pip install pandas 

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 4.7 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.0 MB 4.5 MB/s eta 0:00:03
   ---------- ----------------------------- 2.9/11.0 MB 4.7 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/11.0 MB 4.7 MB/s eta 0:00:02
   ------------------ --------------------- 5.0/11.0 MB 4.8 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.0 MB 4.8 MB/s eta 0:00:02
   ------------------------- -------------- 7.1/11.0 MB 4.8 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.0 MB 4.9 MB/s eta 0:00:01
   ---------------------------------- ----- 9.4/11.0 MB 5.0 MB/s eta 0:00:01
   -------------------------------------- - 10.5/11.0 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 5.0 MB/s  0:00:02

   --


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
#!/usr/bin/env python3
import time
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

SEARCH_URL = "https://www.flipkart.com/search?q=mobiles"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9"
}
TIMEOUT = 15

# Keys you care about (case-insensitive match after normalization)
WANTED_KEYS = {
    "Model Number","Model Name","Processor Type","GPU","Hybrid Sim Slot",
    "Processor Core","Primary Clock Speed","Secondary Clock Speed",
    "Graphics PPI","Display Size","Operating System","Resolution",
    "Internal Storage","External Storage","RAM","Primary Camera",
    "Secondary Camera","Battery Capacity","Weight","Depth"
}

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
    r.raise_for_status()
    return BeautifulSoup(r.content, "html.parser")

def find_product_links(search_soup, limit=5):
    links = set()

    # Pattern 1: product card anchors with href like /p/...
    for a in search_soup.select("a[href*='/p/']"):
        href = a.get("href", "")
        if href.startswith("/"):
            links.add("https://www.flipkart.com" + href.split("?")[0])

    # Pattern 2: common card anchors (class names change; keep it generic)
    for a in search_soup.find_all("a"):
        href = a.get("href")
        if href and "/p/" in href:
            if href.startswith("/"):
                links.add("https://www.flipkart.com" + href.split("?")[0])

    # Deduplicate and cap
    return list(links)[:limit]

def normalize_key(s: str) -> str:
    return re.sub(r"\s+", " ", s.strip()).strip(" :").title()

def extract_specs(product_url):
    soup = get_soup(product_url)

    kv = {}

    # Try a semantic table first
    # Flipkart often uses a table where each row holds spec name/value
    for table in soup.find_all("table"):
        for tr in table.find_all("tr"):
            tds = tr.find_all(["td", "th"])
            if len(tds) >= 2:
                k = normalize_key(tds[0].get_text(" ", strip=True))
                v = tds[1].get_text(" ", strip=True).strip()
                if k and v:
                    kv[k] = v

    # Fallback: definition-list style blocks or list items
    # Find adjacent elements that look like key:value
    # Keys (bold/first cell) + values (next cell/div/span)
    if not kv:
        rows = soup.select("li, div")  # broad net
        for i in range(len(rows)-1):
            k = normalize_key(rows[i].get_text(" ", strip=True))
            v = rows[i+1].get_text(" ", strip=True)
            if len(k) <= 40 and len(v) <= 200 and ":" not in k and k and v:
                # Heuristic: realistic spec keys
                if any(token in k.lower() for token in [
                    "model", "processor", "gpu", "sim", "clock", "graphics",
                    "display", "operating system", "resolution", "storage",
                    "ram", "camera", "battery", "weight", "depth"
                ]):
                    kv.setdefault(k, v)

    # Filter to desired keys (case-insensitive title-cased comparison)
    filtered = {}
    wanted_norm = {normalize_key(k) for k in WANTED_KEYS}
    for k, v in kv.items():
        if normalize_key(k) in wanted_norm:
            filtered[normalize_key(k)] = v

    # Add product title as a helpful extra column if present
    title = soup.find("span", {"class": re.compile(r".*")})
    page_title = soup.find("h1") or soup.find("span", attrs={"class": re.compile("B_NuCI|yhB1nd|.*")})
    product_name = (page_title.get_text(" ", strip=True) if page_title else "").strip()
    if product_name:
        filtered.setdefault("Product Title", product_name)

    filtered.setdefault("URL", product_url)
    return filtered

def main():
    search_soup = get_soup(SEARCH_URL)
    product_links = find_product_links(search_soup, limit=5)  # adjust as needed

    rows = []
    for url in product_links:
        try:
            specs = extract_specs(url)
            rows.append(specs)
            time.sleep(1.5)  # be polite
        except requests.HTTPError as e:
            print(f"HTTP error for {url}: {e}")
        except Exception as e:
            print(f"Failed to parse {url}: {e}")

    if not rows:
        print("No products parsed.")
        return

    # Union of all keys seen so far to build a consistent DataFrame
    all_keys = set().union(*(row.keys() for row in rows))
    # Ensure wanted keys appear first for readability
    ordered_cols = [normalize_key(k) for k in WANTED_KEYS if normalize_key(k) in all_keys]
    # Add the rest (including Product Title, URL)
    ordered_cols += [k for k in all_keys if k not in ordered_cols]

    df = pd.DataFrame(rows)[ordered_cols]
    print(df.to_string(index=False))

if __name__ == "__main__":
    main()


Processor Core   Depth Battery Capacity    Model Name  Ram  Secondary Camera      Processor Type   Primary Camera   Weight              Gpu Internal Storage   Model Number Operating System Graphics Ppi Secondary Clock Speed          Resolution Primary Clock Speed         Display Size Hybrid Sim Slot                                               Product Title                                                                                       URL
     Octa Core 8.22 mm         5160 mAh        C75 5G 4 GB  5MP Front Camera         4s Gen 2 5G 50MP Rear Camera 205.39 g              NaN           128 GB      MZB0KNZIN       Android 14          NaN               1.8 GHz   1650 x 720 Pixels               2 GHz 17.48 cm (6.88 inch)              No                 POCO C75 5G (Aqua Bliss, 128 GB) (4 GB RAM)                 https://www.flipkart.com/poco-c75-5g-aqua-bliss-128-gb/p/itmc8ac71aacc115
     Octa Core  8.6 mm         7000 mAh  g57 power 5G 8 GB  8MP Front Camera            6s Gen 4  